ChemFlow - Computational Chemistry is great again
# ChemFlow - Tutorial 1 - alpha-Thrombin redock.

Diego Enry Barreto Gomes[1,2,3], Cedric Bouysset[1], Marco Cecchini[1]
    1 - Universite de Strasbourg  
    2 - Instituto Nacional de Metrologia, Qualidade e Tecnologia  
    3 - Capes Foundation.  
<p>Chem<i>Flow</i> is an integrated tool designed to predict binding free energies of macromolecular target with a compound library (aka Virtual Screening), through a simple command line interface (and an optional GUI)


## Requirements to run this Jupyter Notebook:
* A mid-2018+ updated version of anaconda3, it will already contain all packages you need. Just make sure you have <b>seaborn version 0.90</b> (updated in July 2018)
* A connetiction to the internet
* <b>Chem<i>Flow</i></b> properly installed.

## Demonstration 1 - Benchmark.
To demonstrate Chem<i>Flow</i> features we designed this Jupyter notebook to interactively run the simulations and analysis. The experiment we are running bellow is to evaluate how different protocols influence the quality of a virtual screening protocol to distinguish <b>active</b> from <b>inactive</b> compounds to a protein target.
    To perform this experiment and be able to accuratelly evaluate the results, we selected the <b>alpha-thrombin</b>, a small protein without many missing residues and with plenty of structural data and curated data binding affinity database, such as the <b>Database of Usefull Decoys - Enhaced</b> (DUD-E). Refer to [ DUD-E ](http://wiki.bkslab.org/index.php/DUDE "DUD-E") website for further info.
    
* Goals:
    * Demonstrate a ChemFlow protocol distinguish active from inactive compounds.
    * Investigate if rescoring can improve pose prediction
    * Investigate if the scoring/energy functions acurately rank the compounds.

* Dataset:
    * Alpha-thrombin (1DWC crystal)
    * 14 ligands from crystal structures
    * DUD-E provides a number of actives and decoys.
    
* Features:
    1. Convert smiles to 3D structure
    2. Assign charges
    3. Dock compounds to it's receptor
    4. Within the 10 
    
    
### Provided files to run this tutorial

| 1DWC.pdb              | Original PDB                                   |
| --- | --- | 
| receptor.pdb          | Receptor prepared with pdb4amber and --reduce. |
| receptor.mol2         | Receptor prepared using SPORES.                |
| reference_ligand.pdb  | Ligand from 1DWC crystal structure.            |
| reference_ligand.mol2 | converted with openbabel.                      |
| ligands.smi           | b1-b7 ligands.                                 |
| ligands_crystal.smi   | 1D3D 1D3P 1D3Q 1D3T 1DWB 1DWC 1DWD             |
| decoys.smi            | decoys for a-thrombin, from DUD-E              |


>b1 comes from the 1DWC crystal, while b2-7 were build up manually by dgomes.

## Step 1: Convert SMILES into 3D structure
ChemFlow implements RDKit to handle 3D structure generation from SMILES, through the state-of-the art [ETKDG](10.1021/acs.jcim.5b00654 "ETKDG") method. RDKit does not write to <b> mol2</b> format, so in sequence you should make a .mol2 file using babel.

### Prepare ligands with known binding affinities.
For the first set of ligands we'll convert b1 to b7 compounds build manually (from an undisclosed article,blame diego).

>python $(which SmilesTo3D.py) -i ligands.smi -o ligands.sdf --hydrogen -v  
>babel -isdf ligands.sdf -omol2 ligands.mol2

### Prepare ligands from crystal structures.
For the second set, with ligands from crystal structures, we also have the affinities. We superimposed 1DWC 1DWB 1DWD 1D3D 1D3P 1D3Q 1D3T (1DWC as reference) and saved all ligands as <b>ligands_crystal.mol2</b>. Hydrogen atoms were added using SPORES (from PLANTS). (SPORES_64bit --mode complete)

### Provide some decoys
Finaly we selected number of thrombin decoys from [DUD-E](http://dude.docking.org/targets/thrb).
Please download with the command below. To be fast, we'll be working with only the first 14.

>wget http://dude.docking.org/targets/thrb/decoys_final.ism  
>head -n 14  decoys_final.ism > decoys.smi

[ WARNING ] You must always be <b>data-aware</b>, for instance, on DUD-E the "field separator" for the text file is a SPACE instead of "\t" (TAB), so you must specify it in SmilesTo3D.py.


>python $(which SmilesTo3D.py) -i decoys.smi -o decoys.sdf --hydrogen -v -d " "  
>babel -isdf decoys.sdf -omol2 decoys.mol2

### Merge all into a single mol2 file: "compounds.mol2"
>cat ligands.mol2 ligands_crystal.mol2 decoys.mol2 > compounds.mol2

We're ready to dock !

## Step 2: Set the center coordinates for the binding pocket

ChemFlow ships a tool to compute optimal box origin and dimensions provided a known ligand. Use the reference ligand to compute the center for docking. You may skip this step if you want to provide the coordinates manually.

For PLANTS it's enough to have only the center.
>python $(which bounding_shape.py) reference_ligand.mol2 --sphere 8.0

For VINA you need the center AND the lenghts of X Y and Z.

>python $(which bounding_shape.py) reference_ligand.mol2 --box 8.0


You should obtain:  
PLANTS: 31.501 13.741 24.359 15.500   
  VINA: 31.501 13.741 24.359 11.832 14.960 12.710

## Step 3: Run DockFlow to predict the docking poses.

To demonstrate **DockFlow** we'll run it with **three** sets of ligands, some of which we only know the binding
affinity (7 compounds), second we know both the affinity and crystal structure (7 compounds)_ and third a set of decoys (14 compounds) All these scenarios will be used in the report different features. In the first place, we'll confront the 14 actives with the 14 decoys and evalute the classification (active/inactive) done by the scoring function from each docking program. Then using the crystal structures we'll evaluate the accuracy of each docking program to produce docking poses near the native one (**docking power**), finally.

Then we'll evaluate the quality of the scoring functions to rank the docking poses (**ranking power**) which will be latter compared with **ScoreFlow**
results together with the **scoring power** which will measure how well it will rank *compounds* against each other.

Let's do it locally:
Run DockFlow for each set of ligands.

* Using plants: ( -sf chemplp,plp,plp95 - chemplp is the default)
>DockFlow -p tutorial --protocol plants -r receptor.mol2 -l compounds.mol2 --center 31.50 13.74 24.36 --radius 20

* Using vina: ( -sf vina )
>DockFlow -p tutorial --protocol vina   -r receptor.mol2 -l compounds.mol2 --center 31.50 13.74 24.36 --size 11.83 14.96 12.71 -sf vina

### Running on HPC
If you are logget to a cluster/supercomputer, you may profit from the HPC resources using <b>--slurm</b> or <b>--pbs</b> flags accordingly. :) ChemFlow will auto<b>magic</b>ally distribute the jobs for you after you answer some questions.
>DockFlow -p tutorial --protocol plants -r receptor.mol2 -l compounds.mol2  --center 31.50 13.74 24.36 --radius 20 --pbs --overwrite


>DockFlow -p tutorial --protocol plants -r receptor.mol2 -l compounds.mol2  --center 31.50 13.74 24.36 --radius 20 --slurm --overwrite


## If all goes right, you should see a summary like this:
![DockFlow Summary](images/DockFlow_summary.png)

## Input validation, checkpointing & resume.
ChemFlow validates the input and generally warns you if something is missing. In addition it checkpoints the calculations and will resume from where it stopped, in case you run out of computer time, system crash, power failure or any other reason. If you want to rerun calculations on the same project+protocol you may choose <b>--overwrite</b>.

Give it a try ! Start a new docking protocol named "test", wait for a while and interrupt. Then relaunch with the same command.

>DockFlow -p tutorial --protocol test -r receptor.mol2 -l compounds.mol2 --center 31.50 13.74 24.36 --radius 20

Interrupt (Cntrl+C), then launch again to see it resuming.

>DockFlow -p tutorial --protocol test -r receptor.mol2 -l compounds.mol2 --center 31.50 13.74 24.36 --radius 20


## Relevant ChemFlow flags so far.
| Flag | Description |
| --- | --- |
| -p,--project | Project name |
| --protocol   | Current protocol |
| -r | receptor filename (.mol2)  | 
| -l | ligand filename (.mol2) | 
| --center | Docking center (XYZ |
| --radius | Search radius |
| --size | Search box dimentions |
|--slurm| Use the Slurm job Scheduller|
|--pbs| Use the PBS job Scheduller| 
|--overwrite| Overwrite (otherwise ChemFlow will resume calculations) |



## Step 4: Postprocess all the results
Docking generates several files for each complex and the big issue here is that each docking program organizes their results their own way.
To organize and standardize all results to the ChemFlow standands you should <b>--postprocess</b>.  Here, we decided to keep only the best 3 poses for each ligand (-n 3).

Each energy and structure will be extracted from it's folder and written into a single Structure file (docked_ligands.mol2) and a single Energy file (DockFlow.csv) containing the selected structures. 


>DockFlow -p tutorial --protocol plants -r receptor.mol2 -l compounds.mol2 --postprocess --overwrite -n 3

>DockFlow -p tutorial --protocol vina   -r receptor.mol2 -l compounds.mol2 --postprocess -sf vina --overwrite -n 3

Answer some questions...
![DockFlow postprocess](images/DockFlow_postprocess.png)

And you'll get an nicelly organized folder with your <b>docked_ligands.mol2</b>
![DockFlow postprocess2](images/DockFlow_postprocess2.png)

## Step 5: Run ScoreFlow to rescore the previous doking poses (best 3 for each ligand)
Here, we only keep on with plants results (tutorial.chemflow/DockFlow/plants/receptor/docked_ligands.mol2).

Rescoring using MMGBSA method:
Currently we implemented 

### Locally:
ScoreFlow -p tutorial --protocol mmgbsa -r receptor.pdb -l tutorial.chemflow/DockFlow/plants/receptor/docked_ligands.mol2 -sf mmgbsa --overwrite --bcc

ScoreFlow -p tutorial --protocol mmgbsa_md       -r receptor.pdb -l tutorial.chemflow/DockFlow/plants/receptor/docked_ligands.mol2 -sf mmgbsa --md --overwrite


### Relevant ScoreFlow flags
| Flag | Description |
| --- | --- | 
| -sf |  Scoring Function |
| --md | Molecular Dynamics simulation | 
| --water | Solvate the system and add counter-ions | 
| --bcc | AM1-BCC charges | 
| --resp | RESP charges (requires Gaussian 09+) |  

For each of these commands you will be asked:

* Are you sure you want to OVERWRITE? > y
* Continue? > y

Same as for DockFlow, if you have access to a cluster, use the --slurm or --pbs flag.

Using a pbs cluster:
++++++++++++++++++++

>ScoreFlow -p tutorial --protocol mmgbsa          -r receptor.pdb -l tutorial.chemflow/DockFlow/plants/receptor/docked_ligands.mol2 -sf mmgbsa              --pbs --overwrite

>ScoreFlow -p tutorial --protocol mmgbsa_md       -r receptor.pdb -l tutorial.chemflow/DockFlow/plants/receptor/docked_ligands.mol2 -sf mmgbsa --md         --pbs --overwrite
   

Step 6: Postprocess the results
-------------------------------
When you are done, you can postprocess (--postprocess) the results. This will generate a single **ScoreFlow.csv** file, containing the rescoring energies.

>ScoreFlow -p tutorial --protocol mmgbsa          -r receptor.pdb -l tutorial.chemflow/DockFlow/plants/receptor/docked_ligands.mol2 -sf mmgbsa --postprocess

>ScoreFlow -p tutorial --protocol mmgbsa_md       -r receptor.pdb -l tutorial.chemflow/DockFlow/plants/receptor/docked_ligands.mol2 -sf mmgbsa --postprocess
